# BindCraftRun: Local Binder Design Starter
Use this notebook to configure and launch BindCraft locally. It assumes your input structures live in `../InputTargets/` and that results should be written under `../Results/`.

# Changelog

28 Dec 2025: 
* Changed colabdesign_utils.py so that min_trajectory_plddt can be set to specify threshold at which "Trajectory starting confidence low (pLDDT < {}), skipping analysis and MPNN optimisation" will execute
* Changed bindcraft.py to improve AA rejection logic when banned AA is passed to MPNN


## Prerequisites
- BindCraft repository cloned and `install_bindcraft.sh` completed.
- Notebook running inside the `BindCraft` conda environment created by the installer.
- AlphaFold2 weights downloaded during installation (default under `params/`).
- CUDA-compatible NVIDIA GPU with drivers matching the selected CUDA runtime.
- Target PDB files available in `../InputTargets/` (adjust the variables below if you store them elsewhere).

In [1]:
from pathlib import Path
import json
import subprocess
import shlex
import os

BINDCRAFT_ROOT = Path(r'/mnt/e/Code/BindCraft').resolve()
BINDCRAFT_SCRIPT = BINDCRAFT_ROOT / 'bindcraft' / 'bindcraft.py'

# Configure JAX environment variables for GPU memory management (Try uncommenting to reduce OOM issues)
# Prevent JAX from preallocating all GPU memory at startup
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
# Limit JAX to use at most 50% of available GPU memory
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.50"
# Disable TensorFlow unified memory to reduce memory pressure
# os.environ["TF_FORCE_UNIFIED_MEMORY"] = "0"
# Use default memory allocator for XLA Python client
# os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "default"


print(f"BindCraft root: {BINDCRAFT_ROOT}")
if 'CONDA_DEFAULT_ENV' in os.environ:
    print(f"Active conda env: {os.environ['CONDA_DEFAULT_ENV']}")
else:
    print('CONDA_DEFAULT_ENV not set; confirm you activated the BindCraft environment.')


BindCraft root: /mnt/e/Code/BindCraft
Active conda env: BindCraft


In [2]:
# Confirm the GPU that BindCraft will see
try:
    subprocess.run(['nvidia-smi'], check=True)
except FileNotFoundError:
    print('nvidia-smi not found. Ensure the NVIDIA drivers are installed and visible in this environment.')
except subprocess.CalledProcessError as exc:
    print('nvidia-smi failed with exit code', exc.returncode)


Mon Dec 29 18:12:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.07             Driver Version: 581.80         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   42C    P8             19W /  285W |    1365MiB /  12282MiB |     20%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Configure target and output paths
Update the variables below to point at your desired settings file, target PDB, and output directory. The defaults use the example `8Z8L.pdb` located in `../InputTargets/`.

In [ ]:
# Core project defaults for a quick CCR4 phage-display run
BINDCRAFT_ROOT = Path(r'/mnt/e/Code/BindCraft').resolve()
BINDCRAFT_SCRIPT = BINDCRAFT_ROOT / 'bindcraft' / 'bindcraft.py'
SETTINGS_DIR = BINDCRAFT_ROOT / 'bindcraft' / 'settings_target'
FILTERS_DIR = BINDCRAFT_ROOT / 'bindcraft' / 'settings_filters'
ADVANCED_DIR = BINDCRAFT_ROOT / 'bindcraft' / 'settings_advanced'

#P51679_ccr4_phage
DESIGN_NAME = 'test'
FILTERS_JSON = FILTERS_DIR / 'peptide_relaxed_filters_phage.json'
ADVANCED_JSON = ADVANCED_DIR / 'P51679_ccr4_phage.json'

TARGETS_DIR = (BINDCRAFT_ROOT / 'InputTargets').resolve()
TARGET_PDB_NAME = 'MIN_ANCHOR_AF-P51679-F1-model_v6_trimmed_bundle.pdb'
TARGET_CHAIN = 'A'
HOTSPOT_RESIDUES = None
BINDER_LENGTH_RANGE = [18, 20]
TARGET_PDB = (TARGETS_DIR / TARGET_PDB_NAME).resolve()
if not TARGET_PDB.exists():
    raise FileNotFoundError(f'Target PDB {TARGET_PDB} not found. Update TARGET_PDB_NAME or TARGETS_DIR.')

RESULTS_BASE = (BINDCRAFT_ROOT / 'Results').resolve()
RESULTS_BASE.mkdir(parents=True, exist_ok=True)
DESIGN_OUTPUT = RESULTS_BASE / DESIGN_NAME
DESIGN_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f'Design name       : {DESIGN_NAME}')
print(f'Filters file      : {FILTERS_JSON}')
print(f'Advanced file     : {ADVANCED_JSON}')
print(f'Targets directory : {TARGETS_DIR}')
print(f'Target PDB        : {TARGET_PDB}')
print(f'Target chain      : {TARGET_CHAIN}')
print(f'Hotspot residues  : {HOTSPOT_RESIDUES}')
print(f'Design output dir : {DESIGN_OUTPUT}')


Design name       : test
Filters file      : /mnt/e/Code/BindCraft/bindcraft/settings_filters/peptide_relaxed_filters_phage.json
Advanced file     : /mnt/e/Code/BindCraft/bindcraft/settings_advanced/P51679_ccr4_phage.json
Targets directory : /mnt/e/Code/BindCraft/InputTargets
Target PDB        : /mnt/e/Code/BindCraft/InputTargets/AF-P51679-F1-model_v6_trimmed_bundle.pdb
Target chain      : A
Hotspot residues  : None
Design output dir : /mnt/e/Code/BindCraft/Results/test


### Create or reuse a dedicated settings file
Set `CREATE_SETTINGS_COPY = True` the first time you run for a new project. The template (`settings.json`) will be copied to `settings_target/<design name>.json`, with `design_path`, `binder_name`, and `starting_pdb` updated to match your local folders. If the file already exists it will be reused unless you enable `OVERWRITE_SETTINGS`.

In [4]:
# Use existing settings file without template generation
RUN_SETTINGS = SETTINGS_DIR / 'P51679_ccr4_phage.json'
print(f'Using settings file: {RUN_SETTINGS}')


Using settings file: /mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_ccr4_phage.json


In [5]:
# Print summary of advanced settings

print('Advanced settings file in use:', ADVANCED_JSON)
with ADVANCED_JSON.open() as fh:
    advanced_cfg = json.load(fh)

summary_keys = ['number_of_final_designs','max_trajectories','min_trajectory_plddt','enable_mpnn','use_multimer_design','enable_rejection_check','force_reject_AA','omit_AAs','design_models_override','sample_models','num_recycles_design','num_recycles_validation','soft_iterations','temporary_iterations','hard_iterations','greedy_iterations','inter_contact_distance','inter_contact_number','weights_helicity','sampling_temp','max_mpnn_sequences','num_seqs','start_monitoring','save_design_animations','save_design_trajectory_plots']
for key in summary_keys:
    value = advanced_cfg.get(key, '<not present>')
    print(f'{key:>28}: {value}')


Advanced settings file in use: /mnt/e/Code/BindCraft/bindcraft/settings_advanced/P51679_ccr4_phage.json
     number_of_final_designs: 5
            max_trajectories: False
        min_trajectory_plddt: 0.2
                 enable_mpnn: True
         use_multimer_design: True
      enable_rejection_check: False
             force_reject_AA: False
                    omit_AAs: C
      design_models_override: <not present>
               sample_models: False
         num_recycles_design: 1
     num_recycles_validation: 1
             soft_iterations: 75
        temporary_iterations: 45
             hard_iterations: 5
           greedy_iterations: 15
      inter_contact_distance: 14.0
        inter_contact_number: 8
            weights_helicity: 0.3
               sampling_temp: 0.3
          max_mpnn_sequences: 6
                    num_seqs: 10
            start_monitoring: 0
      save_design_animations: False
save_design_trajectory_plots: False


In [ ]:
#Use a non-Qt backend. Switch matplotlib to Agg before importing anything that touches Qt:
#Solve dependency conflicts when running in Jupyter notebooks


import matplotlib
matplotlib.use("Agg")  # do this right after launching the kernel


## Launch BindCraft
Set `LAUNCH_RUN = True` when you are ready. The command runs in blocking mode; expect long runtimes for realistic designs. Interrupt the cell if you need to stop the run.

In [7]:
LAUNCH_RUN = True
additional_args = []

env = os.environ.copy()
conda_prefix = env.get('CONDA_PREFIX')
if not conda_prefix:
    raise RuntimeError('Activate the BindCraft environment before launching.')
existing_ld = env.get('LD_LIBRARY_PATH', '')
if existing_ld:
    env['LD_LIBRARY_PATH'] = '{}/lib:{}'.format(conda_prefix, existing_ld)
else:
    env['LD_LIBRARY_PATH'] = '{}/lib'.format(conda_prefix)

cmd = [
    'python', '-u', str(BINDCRAFT_SCRIPT),
    '--settings', str(RUN_SETTINGS),
    '--filters', str(FILTERS_JSON),
    '--advanced', str(ADVANCED_JSON),
] + additional_args

print('Command to run:')
print(' '.join(shlex.quote(part) for part in cmd))
print('LD_LIBRARY_PATH:', env['LD_LIBRARY_PATH'])

if LAUNCH_RUN:
    result = subprocess.run(cmd, cwd=BINDCRAFT_ROOT, env=env, check=False)
    print(f'BindCraft finished with exit code {result.returncode}')
else:
    print('Set LAUNCH_RUN = True to start the design run.')


Command to run:
python -u /mnt/e/Code/BindCraft/bindcraft/bindcraft.py --settings /mnt/e/Code/BindCraft/bindcraft/settings_target/P51679_ccr4_phage.json --filters /mnt/e/Code/BindCraft/bindcraft/settings_filters/peptide_relaxed_filters_phage.json --advanced /mnt/e/Code/BindCraft/bindcraft/settings_advanced/P51679_ccr4_phage.json
LD_LIBRARY_PATH: /root/miniconda3/envs/BindCraft/lib
Available GPUs:
NVIDIA GeForce RTX 4070 Ti1: gpu
┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details    

KeyboardInterrupt: 

## Inspect recent outputs
Use the helper below to list the most recently modified files under your design directory once a run has started producing results.

In [ ]:
from datetime import datetime

if DESIGN_OUTPUT.exists():
    tracked_files = sorted(
        (path for path in DESIGN_OUTPUT.glob('**/*') if path.is_file()),
        key=lambda p: p.stat().st_mtime,
        reverse=True,
    )[:10]
    if tracked_files:
        print('Latest files:')
        for path in tracked_files:
            timestamp = datetime.fromtimestamp(path.stat().st_mtime).isoformat(sep=' ', timespec='seconds')
            print(f'{timestamp} :: {path.relative_to(DESIGN_OUTPUT)}')
    else:
        print('No files in the design output directory yet.')
else:
    print('Design output directory does not exist. Run the configuration cells first.')


Latest files:
2025-12-27 22:02:23 :: failure_csv.csv
2025-12-27 22:02:23 :: final_design_stats.csv
2025-12-27 22:02:23 :: mpnn_design_stats.csv
2025-12-27 22:02:23 :: trajectory_stats.csv
